In [ ]:
import tables
import pandas as pd
import numpy as np
def read_data_tables(data_path, cycle):
        try:
            with tables.open_file(data_path, mode='r') as file:
                data_node = file.get_node(f"/Cycle{cycle:04d}/block0_values")
                data_array = data_node[:]  # Exclude the first column
                columns_node = file.get_node(f"/Cycle{cycle:04d}/block0_items")
                columns = columns_node[:].astype(str) # Exclude the first column name
                return pd.DataFrame(data_array, columns=columns)
        except Exception as e:
            print(e)
            return None
def read_capacity_tables(data_path, cycle):
        try:
            with tables.open_file(data_path, mode='r') as file:
                data_node = file.get_node(f"/capacity{cycle:04d}/block0_values")
                data_array = data_node[:]  # Exclude the first column
                columns_node = file.get_node(f"/capacity{cycle:04d}/block0_items")
                columns = columns_node[:].astype(str) # Exclude the first column name
                return pd.DataFrame(data_array, columns=columns)
        except Exception as e:
            print(e)
            return None
    # Function to read SoH using PyTables
def read_soh_tables(data_path):
        try:
            with tables.open_file(data_path, mode='r') as file:
                soh_node = file.get_node("/Info/block0_values")
                soh_array = soh_node[:]  # Exclude the first column
                columns_node = file.get_node("/Info/block0_items")
                columns = columns_node[:].astype(str) # Exclude the first column name
                soh_data = soh_array[:, columns == 'real_capacity'].flatten()
                soh_data = soh_data / soh_data[0]
                return pd.Series(soh_data)
        except Exception as e:
            return None

cycle = 0
file_path = "Synthetic_data/episode_ffa646b4-319f-4e87-bfdd-24270224424d.result"

def get_cycle_data(file_path, cycle):
    data = read_data_tables(file_path, cycle)
    capacity_test = read_capacity_tables(file_path, cycle)
    capacity_test = capacity_test.drop("SoC", axis = 1)
    soh_data = read_soh_tables(file_path)
    
    if data is None or soh_data is None or len(data) == 0:
        raise IndexError(
            f"No valid data found for file {file_path}, cycle {cycle}"
        )
    
    
    if data.empty:
        raise IndexError(
            f"Cycle data is empty for file {file_path}, cycle {cycle}"
        )
    
    soh_value = soh_data[cycle] if cycle < len(soh_data) else None
    
    return data, capacity_test, soh_value

cycle_data, capacity_test, soh_value = get_cycle_data(file_path, cycle)


In [ ]:
# Display the structure and summary of cycle_data, capacity_test, and soh_value
print("Cycle Data:")
print(cycle_data.info())
print(cycle_data.head())

print("\nCapacity Test Data:")
print(capacity_test.info())
print(capacity_test.head())

print("\nState of Health (SoH) Value:")
print(soh_value)

In [ ]:
import plotly.express as px

fig = px.line(cycle_data, x='relative_time', y='V', title='Voltage vs Time', labels={'relative_time': 'Time', 'V': 'Voltage'})
fig.show()

In [ ]:
fig = px.line(cycle_data, x='relative_time', y='C', title='Voltage vs Time', labels={'relative_time': 'Time', 'V': 'Voltage'})
fig.show()

In [ ]:
fig = px.line(cycle_data, x='relative_time', y='SoC', title='Voltage vs Time', labels={'relative_time': 'Time', 'V': 'Voltage'})
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(capacity_test, x='relative_time', y='V', title='Voltage vs Time', labels={'relative_time': 'Time', 'V': 'Voltage'})
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Add Voltage trace
fig.add_trace(
    go.Scatter(x=cycle_data['relative_time'], y=cycle_data['V'], name='Voltage', yaxis='y1')
)

# Add Current trace
fig.add_trace(
    go.Scatter(x=cycle_data['relative_time'], y=cycle_data['C'], name='Current', yaxis='y2')
)

# Create axis objects
fig.update_layout(
    title='Voltage and Current vs Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Voltage', side='left'),
    yaxis2=dict(title='Current', overlaying='y', side='right')
)

fig.show()